# Thermograms

In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import itertools
import numpy as np
from scipy.signal import argrelmin
from sklearn.metrics import auc

In [2]:
list_files = glob.glob("C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\**[!Res].txt") 
print(list_files)
Res_list_files = glob.glob("C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\*Res.txt") 
print(Res_list_files)

list_files_Res_dict = dict(zip(list_files,Res_list_files))   

out_path = "C:\\Users\\DominicHeslinRees\\Pictures\\black_carbon\\"

['C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0130.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0131.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0201.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0212.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0213.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0215.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0423.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0425.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0503.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0504.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0514.txt', 'C:\\Users\\DominicHeslinRees\\Documents\\Black carbon\\Zeppelin\\2007\\0822.txt', 'C:

The area at each point along the thermogram curve minus the baseline is multiplied against the calibration response to determine the carbon.

The automated OC/EC split point is calculated from where the laser absorbance in the oxidizing phase of the EC analysis matches the initial absorbance measured when the sample was first inserted into the oven. Carbon observed before the split point is considered organic carbon and carbon after the split is considered elemental carbon. 

The analysis program can batch process all of the sample data sets in a file without further review. It is recommended that the analyst study each individual data set as it is processed to look for any anomalies.

If necessary, the OC/EC split point can be manually set (reassigned) by moving the cursor to the desired point. The software will calculate the split based on that point. This decision is based on the analyst’s experience and information about the sample.

In [3]:
def find_indexs(file):
    file_info = open(file, 'r')

    FID1_indexes = []
    sample_indexes = []

    lines = file_info.readlines()
    file_size = len(lines)
    
    for i, line in enumerate(lines): 
        
        if 'FID1' in line:
            #extract line index for lines that contain FID1
            FID1_indexes.append(i + 1)
        elif 'Sample' in line:
            sample_indexes.append(i + 1)   
    
    sample_IDs = [x.replace('\n', '') for i, x in enumerate(lines) if i in sample_indexes]
        
    return file_size,FID1_indexes, sample_indexes, sample_IDs

In [4]:
for file in list_files:
    print(file)
    
    file_size, FID1_indexes, sample_indexes, sample_IDs = find_indexs(file) #raw data file
    sample_indexes = sample_indexes[1:]   #indexs for the samples when they start so each blocked can be read in    
    sample_indexes.insert(len(sample_indexes), file_size)
    
    print("Number of samples: "+str(len(sample_indexes)))    
    
    Res_file = list_files_Res_dict[file]  #need to match each block with the results file using the sample ID
    data_Res = pd.read_csv(Res_file, sep="\t", skiprows=4, encoding= 'unicode_escape', index_col=False)
    
    
    for i in range(len(FID1_indexes)):    
        print(i)
        print("Index:")
        start_row = FID1_indexes[i]
        print("Start row: "+str(start_row))
        num_row = sample_indexes[i]-start_row-2 
        print("Number row: "+str(num_row))       
        print(file)
        data = pd.read_csv(file, sep="\t", skiprows=start_row, nrows=num_row, encoding= 'unicode_escape', 
            names=["FID1", "FID2", "front_temperature", "laser", "laserlow", "pressure", "mode", "unknown"], index_col=False)

        FID2_start = data['FID2'][0]
        print(FID2_start)
        FID2_min = data['FID2'].min()
        print(FID2_min)
        idx = data.loc[lambda data: data['FID2'] == FID2_min].index
        print(idx)
        
        data['FID2'] = data['FID2'] - FID2_start
        
        
        
        filter_name =  sample_IDs[i]
        cal_area = data_Res.loc[data_Res['Sample ID'] == str(filter_name), 'calibration area'].iloc[0]

        data_Res = data_Res[pd.to_numeric(data_Res['Split time(sec)'], errors='coerce').notnull()]      
        split_time = data_Res.loc[data_Res['Sample ID'] == str(filter_name), 'Split time(sec)'].iloc[0]
        split_time = int(split_time)
        print("Split time: "+str(split_time))
        
        
        cols = ['OC(ug/sq cm)', 'EC(ug/sq cm)', 'CC(ug/sq cm)', 'TC(ug/sq cm)','Pyrol C ug/sqcm']
        
        data_Res[cols] = data_Res[cols].apply(pd.to_numeric, errors='coerce', axis=1)
        
        #radius = 1.5
        #filter_area = radius**2*np.pi
        #data_Res['OC'] = data_Res['OC(ug/sq cm)']*filter_area/data_Res['Sample Volume']
        
        OC = data_Res.loc[data_Res['Sample ID'] == str(filter_name), r'OC(ug/sq cm)'].iloc[0]
        OC = float(OC)

        EC = data_Res.loc[data_Res['Sample ID'] == str(filter_name), r'EC(ug/sq cm)'].iloc[0]
        EC = float(EC)        
   
        PC = data_Res.loc[data_Res['Sample ID'] == str(filter_name), r'Pyrol C ug/sqcm'].iloc[0]
        PC = float(PC)
        
        Calconst = data_Res.loc[data_Res['Sample ID'] == str(filter_name), r'CalConst'].iloc[0]
        Calconst = float(Calconst)
        
        filter_name = filter_name + ' ' + file[-8:-4]
         
        plot_thermogram(data, filter_name, cal_area, split_time, OC, EC, PC, Calconst, FID2_min, FID2_start)

C:\Users\DominicHeslinRees\Documents\Black carbon\Zeppelin\2007\0130.txt
Number of samples: 11
0
Index:
Start row: 23
Number row: 780
C:\Users\DominicHeslinRees\Documents\Black carbon\Zeppelin\2007\0130.txt
121
120
Int64Index([5, 10, 15, 26, 27, 28, 29, 34, 36, 39], dtype='int64')
Split time: 676


NameError: name 'plot_thermogram' is not defined

In [ ]:
data_Res.columns

In [ ]:
OC = OC(ug/sq cm) * Sampled filter area cm2 / Sampled volume m3

In [ ]:
data_Res

In [ ]:
data = data.iloc[:778,:]

<img src="Example-thermogram-from-analysis-of-OC-and-EC-by-thermaloptical-transmittance-in-a-Sunset.png">

In [ ]:
def calculate_area(time_FID2_dict, idx1, idx2):
    time_FID2_dict_slice = itertools.islice(time_FID2_dict.items(), idx1 , idx2)
    time_FID2_dict_list = list(time_FID2_dict_slice)
    
    res = [[ i for i, j in time_FID2_dict_list ], 
       [ j for i, j in time_FID2_dict_list ]] 
      
    times = res[0]
    FID2 = res[1] 
    
    print(max(FID2))
    print(min(FID2))
    
    print("Lengths: ")
    print(len(FID2))
    
    FID2 = [i if i >= 0 else 0 for i in FID2]

    print(len(FID2))
        
    print(str(idx1)+"//"+str(idx2))
    print('computed AUC using sklearn.metrics.auc: {}'.format(auc(times,FID2)))
    print('computed AUC using np.trapz: {}'.format(np.trapz(FID2, dx = 1)))
    
    Area = auc(times,FID2)
    print(Area)
    return Area

In [ ]:
def plot_thermogram(data, filter_name, cal_area, split_time, OC, EC, PC, Calconst,  FID2_min, FID2_start):
    
    idx_he = data[data['mode'] == 'He'].index[0]
    print("idx He: "+str(idx_he))
    idx_Ox = data[data['mode'] == 'Ox'].index[0]
    print("idx He: "+str(idx_Ox))
    idx_Ch4 = data[data['mode'] == 'CH4+Ox'].index[0]
    print("idx He: "+str(idx_Ch4))

    fig, ax1 = plt.subplots(figsize=(15,8))

    color = 'black'
    
    ax1.set_ylabel('FID2', color=color, fontsize=15) #ax2 is FID2
    ax1.plot(data.index, data['FID2'], 'o', color=color, ms=1)
    ax1.fill(data.index, data['FID2'], alpha=0.5)    
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx() 

    color = 'red'
    
    ax2.set_xlabel('time (s)', fontsize=15)
    ax2.plot(data.index, data['front_temperature'], 'o', color=color, ms=1)
    ax2.plot(data.index, data['laser'], 'o', color=color, ms=1) #temperature        
    ax2.set_ylabel('Front temperature/laser', color=color, fontsize=15) #ax1 is temperature    
    ax2.tick_params(axis='y', labelcolor=color)

    plt.title('Thermogram: '+str(filter_name))

    plt.axvline(x=idx_he, ls=':', c='red', label='He')
    plt.axvline(x=idx_Ox, ls=':', c='blue', label='Ox')
    plt.axvline(x=idx_Ch4, ls=':', c='black', label='CH4+Ox')  
    plt.axvline(x=split_time, ls='-', c='black', label='Split time: '+str(split_time))  
    
    print('Total area under the curve (computed using np.trapz): {}'.format(np.trapz(data['FID2'].values, dx = 1)))
    
    FID2_values = data['FID2'].values
    time = np.linspace(data.index[0],data.index[-1],len(FID2_values))    
    time_FID2_dict = dict(zip(time,FID2_values))     
     
    print("He-split")    
    
    Area_He_split = calculate_area(time_FID2_dict, idx1=idx_he, idx2=split_time)
    print("Split-Ox")   
    Area_split_Ox = calculate_area(time_FID2_dict, idx1=min(idx_Ox, split_time), idx2=max(idx_Ox, split_time))
    
    print("Ox-Ch4")   
    Area_Ox_Ch4 = calculate_area(time_FID2_dict, idx1=idx_Ox, idx2=idx_Ch4)
    print("Ch4-end")   
    Area_Ch4_end = calculate_area(time_FID2_dict, idx1=idx_Ch4, idx2=800)
    
    temp_values = data['front_temperature'].values
    
    local_max_idx = argrelmin(FID2_values)[0] #argrelextrema(FID2_values, np.greater)
    local_max = temp_values[local_max_idx] #temp_values[argrelextrema(FID2_values, np.greater)[0]]
    
    #ax2.plot(local_max_idx, local_max, 'x', ms=10)
    
    ratio = float(Area_Ch4_end)/float(cal_area)*100
    ratio = '{0:.3f}'.format(ratio)
    OC_3sf = '{0:.3f}'.format(OC)
    EC_3sf = '{0:.3f}'.format(EC)
    
    OC_est = Area_He_split/(Calconst*100)
    OC_est_3sf = '{0:.3f}'.format(OC_est)
    
    PC_EC_est = Area_Ox_Ch4/(Calconst*100)
    PC_EC_est_3sf = '{0:.3f}'.format(PC_EC_est)
    
    PC_est = Area_split_Ox/(Calconst*100)
    PC_est_3sf = '{0:.3f}'.format(PC_est)
    
    FID2_min =  '{0:.3f}'.format(FID2_min)
    FID2_start =  '{0:.3f}'.format(FID2_start)
    
    plt.text(idx_he+10, 30000, "FID2 min: "+str(FID2_min)+"\n"+"FID2 start: "+str(FID2_start), fontsize = 10, va="top", ha="left")

    plt.text(idx_he+150, 15000, "Area He-Split (OC):\n"+str(Area_He_split)+r' => '+str(OC_est_3sf), fontsize = 10, va="top", ha="left")    
    plt.text(idx_he+150, 13000, "OC: "+str(OC_3sf)+" ug/sq cm", fontsize = 10, va="top", ha="left")
    plt.text(idx_he+150, 11000, "EC: "+str(EC_3sf)+" ug/sq cm", fontsize = 10, va="top", ha="left")
    
    
    plt.text(idx_Ox+10, 15000, "Area Split-Ox (PC):\n"+str(Area_split_Ox)+'\n' + r'=> '+str(PC_est_3sf), fontsize = 10, va="top", ha="left")
    plt.text(idx_Ch4-100, 15000, "Area Ox-Ch4 (PC+EC):\n"+str(Area_Ox_Ch4)+'\n' + r'=> '+str(PC_EC_est_3sf), fontsize = 10, va="top", ha="right")
    plt.text(idx_Ch4-10, 7000, "Area Ch4-end\n(Calibration):\n"+str(Area_Ch4_end), fontsize = 10, va="top", ha="right")
    plt.text(idx_Ch4-100, 7000, "Calibratio Area\ngiven by file:\n"+str(cal_area), fontsize=10, va="top", ha="right")
    plt.text(idx_Ch4-10, 4000, "Cal const :\n"+str(ratio)+'\ngiven in file: '+str(Calconst), fontsize=10, va="top", ha="right")

    ax1.set_ylim(0,4100)
    #ax2.set_xlim(0,800)
    
    #ax1.vlines(x=split_time, ymin=-100, ymax=5000, ls='-', colors='black', label='Split time: '+str(split_time))
    #ax1.legend()
    
    plt.legend()
    
    plt.savefig(out_path+"theromgram\\Thermogram "+str(filter_name)+".jpeg", dpi=200,bbox_inches='tight')
        
    plt.show()
       
    #fig.tight_layout()  
    
    fig, ax = plt.subplots(figsize=(5,5))
    areas = [(Area_Ox_Ch4-Area_split_Ox), Area_split_Ox, Area_He_split]
    concentrations = [EC, PC, OC]
    print(areas)
    print(concentrations)
    
    ax.plot(areas, concentrations, 'o-', ms=10, mec='black', color='none')
    
    ax.set_ylim(min(concentrations), max(concentrations))
    ax.set_xlim(min(areas), max(areas))
    
    return plt

In [ ]:
for file in list_files:
    print(file)
    
    file_size, FID1_indexes, sample_indexes, sample_IDs = find_indexs(file) #raw data file
    sample_indexes = sample_indexes[1:]   #indexs for the samples when they start so each blocked can be read in    
    sample_indexes.insert(len(sample_indexes), file_size)
    
    print("Number of samples: "+str(len(sample_indexes)))    
    
    Res_file = list_files_Res_dict[file]  #need to match each block with the results file using the sample ID
    data_Res = pd.read_csv(Res_file, sep="\t", skiprows=4, encoding= 'unicode_escape', index_col=False)
    
    
    for i in range(len(FID1_indexes)):    
        print(i)
        print("Index:")
        start_row = FID1_indexes[i]
        print("Start row: "+str(start_row))
        num_row = sample_indexes[i]-start_row-2 
        print("Number row: "+str(num_row))       
        print(file)
        data = pd.read_csv(file, sep="\t", skiprows=start_row, nrows=num_row, encoding= 'unicode_escape', 
            names=["FID1", "FID2", "front_temperature", "laser", "laserlow", "pressure", "mode", "unknown"], index_col=False)

        FID2_start = data['FID2'][0]
        print(FID2_start)
        FID2_min = data['FID2'].min()
        print(FID2_min)
        idx = data.loc[lambda data: data['FID2'] == FID2_min].index
        print(idx)
        
        data['FID2'] = data['FID2'] - FID2_start
        
        
        
        filter_name =  sample_IDs[i]
        cal_area = data_Res.loc[data_Res['Sample ID'] == str(filter_name), 'calibration area'].iloc[0]

        data_Res = data_Res[pd.to_numeric(data_Res['Split time(sec)'], errors='coerce').notnull()]      
        split_time = data_Res.loc[data_Res['Sample ID'] == str(filter_name), 'Split time(sec)'].iloc[0]
        split_time = int(split_time)
        print("Split time: "+str(split_time))
        
        
        cols = ['OC(ug/sq cm)', 'EC(ug/sq cm)', 'CC(ug/sq cm)', 'TC(ug/sq cm)','Pyrol C ug/sqcm']
        
        data_Res[cols] = data_Res[cols].apply(pd.to_numeric, errors='coerce', axis=1)
        
        #radius = 1.5
        #filter_area = radius**2*np.pi
        #data_Res['OC'] = data_Res['OC(ug/sq cm)']*filter_area/data_Res['Sample Volume']
        
        OC = data_Res.loc[data_Res['Sample ID'] == str(filter_name), r'OC(ug/sq cm)'].iloc[0]
        OC = float(OC)

        EC = data_Res.loc[data_Res['Sample ID'] == str(filter_name), r'EC(ug/sq cm)'].iloc[0]
        EC = float(EC)        
   
        PC = data_Res.loc[data_Res['Sample ID'] == str(filter_name), r'Pyrol C ug/sqcm'].iloc[0]
        PC = float(PC)
        
        Calconst = data_Res.loc[data_Res['Sample ID'] == str(filter_name), r'CalConst'].iloc[0]
        Calconst = float(Calconst)
        
        filter_name = filter_name + ' ' + file[-8:-4]
         
        plot_thermogram(data, filter_name, cal_area, split_time, OC, EC, PC, Calconst, FID2_min, FID2_start)

The quantification is done by determining the area under the peaks and the instrument is calibrated with samples of known amounts of carbon (e.g.,sucrose).

The calculation program that is provided by Sunset Laboratory calculates the area under the peaks in the thermogram by subtracting a linear sloped background (baseline). This background is obtained from a number of channels at the start of the thermogram and a number of channels before the CH4 internal calibration peak.